# Library


In [ ]:
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.4 MB/s eta 0:00:00


In [ ]:
!pip install nltk

In [ ]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.1 MB/s eta 0:00:00


In [ ]:
!pip install gensim

In [ ]:
from google_play_scraper import Sort, reviews_all, reviews
import pandas as pd
from datetime import datetime
from IPython.display import clear_output
import time
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt_tab')
from nltk.corpus import stopwords
nltk.download('stopwords')
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tqdm import tqdm
from gensim.models import Word2Vec
import os
from sklearn.model_selection import train_test_split
import numpy as np
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Dense, Embedding, Dropout, Input
from keras.models import Sequential

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.








# Scrapper

In [ ]:
app_package = "com.levelinfinite.sgameGlobal"

result, _ = reviews(
    app_package,
    lang="id",  # Bahasa Indonesia
    country="id",  # Negara Indonesia
    sort=Sort.NEWEST  # Ulasan terbaru
)

df = pd.DataFrame(result)
df.to_csv("ulasan_honor_of_kings.csv", index=False)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              5000 non-null   object        
 1   userName              5000 non-null   object        
 2   userImage             5000 non-null   object        
 3   content               5000 non-null   object        
 4   score                 5000 non-null   int64         
 5   thumbsUpCount         5000 non-null   int64         
 6   reviewCreatedVersion  3780 non-null   object        
 7   at                    5000 non-null   datetime64[ns]
 8   replyContent          1109 non-null   object        
 9   repliedAt             1109 non-null   datetime64[ns]
 10  appVersion            3780 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 429.8+ KB


In [ ]:
df['score'].value_counts()

,count
score,
5,2437
1,1886
2,294
4,247
3,136


In [ ]:
print(df['at'].max())
print(df['at'].min())

2024-07-05 23:47:00
2024-06-24 14:57:00


In [ ]:
df['reviewId'].is_unique

True

In [ ]:
df['userName'].is_unique

False

In [ ]:
duplicate_usernames = df['userName'].value_counts()
duplicate_usernames = duplicate_usernames[duplicate_usernames > 1]
duplicate_usernames

,count
userName,
Muhamad Ridwan,3
Muhammad Rifqi,3
Heri Yanto,2
Daffa,2
Rafi Firdaus,2
Jihan Salsabila,2
Ilham,2
Maulana Saputra,2
Taufik Hidayat,2


In [ ]:
duplicate_contents = df[df['userName'].isin(duplicate_usernames.index)]
duplicate_contents[['userName', 'content']].sort_values(by='userName')

,userName,content
2104,Abi Abi,Merendahkan ml
2194,Abi Abi,Kaga jelas banget dark sistem dari monton pind...
493,Aditya Pratama,sangat bagus saya suka game ini
3390,Aditya Pratama,Kekurangan dari game ini adalah delay skill ya...
4655,Azka,Recall nya cepetin pls
4637,Azka,"overall, game bagus mekanisme bagus dari segal..."
4148,Daffa,Gamenya lumayan tapi tolong jaringan dioptimal...
3595,Daffa,"Sering berenti loading muter tiba tiba, Padaha..."
981,Heri Yanto,Oke banyak Hero gratis
3446,Heri Yanto,lebih parah musuh ya jago2 sedakan tim ku sela...


In [ ]:
# Pilih kolom yang diinginkan
df_clean = df[['reviewId', 'userName', 'content', 'score']]

# Simpan ke file CSV baru
df_clean.to_csv('hok_reviews_clean.csv', index=False)

# Labeling data

In [ ]:
df_clean = pd.read_csv("hok_reviews_clean.csv")

In [ ]:
# Fungsi untuk menambah kolom sentimen secara manual
def add_sentiment(df_clean, start, end):
    sentiments = []

    # Iterasi dari data ke-n hingga ke-m
    for index, row in df_clean.iloc[start:end].iterrows():
        # Menghapus tampilan sebelumnya
        time.sleep(0.5)
        clear_output(wait=True)


        # Menampilkan data saat ini
        print(f"Data {index + 1}")
        print(f"Content: {row['content']}")
        print(f"Score: {row['score']}")

        # Meminta input dari user untuk sentimen
        while True:
            # Menerima input dan memastikan input berupa 0 atau 1
            sentiment = int(input("Input sentimen (0 atau 1): "))
            if sentiment in [1,0,3]:
                sentiments.append(sentiment)
                break
            else:
                print("Input harus berupa 0 atau 1. Silakan coba lagi.")
        print("\n")

    # Menambahkan kolom sentimen ke dataframe
    df_clean.loc[start:end-1, 'sentimen'] = sentiments

# Memanggil fungsi untuk menambahkan sentimen ke data dari urutan 1300 hingga 1400
add_sentiment(df_clean, 4750, 5000)

Data 5000
Content: ok
Score: 4
Input sentimen (0 atau 1): 3




In [ ]:
# Simpan dataset yang sudah ditambahkan kolom sentimen

df_clean.to_csv("HOK_Labeled.csv", index=False)

# Text Preprocessing

In [ ]:
data = pd.read_csv('HOK_Labeled.csv')
data = data[['content','sentimen']]
data = data[data['sentimen'].isin([1, 0])]
data

,content,sentimen
0,Bagussssssssssss,1.0
1,LOSE STREAK MULU PADAHAL MASIH GOLD MUSUH UDAH...,0.0
2,Gamenya bagus dan nggak butuh penyimpanan yg b...,1.0
3,gm nya lebih bagus dari pada ml grafik hd,1.0
4,bagus game nya,1.0
...,...,...
3711,Game nya burik banget,0.0
3712,SAMPAH. GRAFIK DOANG. SKIL ANEH GA GUNA. KESEI...,0.0
3713,Masa dapat team kaya bocil kita yg pelanggaran...,0.0
3714,Gem plgiat,0.0


In [ ]:
data['sentimen'].value_counts()

,count
sentimen,
1.0,1900
0.0,1816


## Lower Casing

In [ ]:
# Melakukan lower casing pada kolom 'content'
data['lower_case'] = data['content'].str.lower()

# Menampilkan hasil
data[['content','lower_case']]

,content,lower_case
0,Bagussssssssssss,bagussssssssssss
1,LOSE STREAK MULU PADAHAL MASIH GOLD MUSUH UDAH...,lose streak mulu padahal masih gold musuh udah...
2,Gamenya bagus dan nggak butuh penyimpanan yg b...,gamenya bagus dan nggak butuh penyimpanan yg b...
3,gm nya lebih bagus dari pada ml grafik hd,gm nya lebih bagus dari pada ml grafik hd
4,bagus game nya,bagus game nya
...,...,...
3711,Game nya burik banget,game nya burik banget
3712,SAMPAH. GRAFIK DOANG. SKIL ANEH GA GUNA. KESEI...,sampah. grafik doang. skil aneh ga guna. kesei...
3713,Masa dapat team kaya bocil kita yg pelanggaran...,masa dapat team kaya bocil kita yg pelanggaran...
3714,Gem plgiat,gem plgiat


## Cleaning

### Menghapus tanda baca, emoticon, angka, dan hashtag (jika ada)

In [ ]:
# Fungsi untuk menghapus tanda baca, emoticon, angka, dan hashtag
def clean_text(text):
    text = re.sub(r'[^\x00-\x7F]+', '', text) # Menghapus emoticon menggunakan regex untuk karakter non-ASCII
    text = re.sub(r'#\w+', '', text) # Menghapus hashtag
    text = re.sub(r'\d+', '', text) # Menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # Menghapus tanda baca
    return text

# Menerapkan fungsi clean_text pada kolom 'lower_case'
data['clean_1'] = data['lower_case'].apply(clean_text)

# Menampilkan beberapa baris pertama untuk memverifikasi
data[['lower_case','clean_1']]


,lower_case,clean_1
0,bagussssssssssss,bagussssssssssss
1,lose streak mulu padahal masih gold musuh udah...,lose streak mulu padahal masih gold musuh udah...
2,gamenya bagus dan nggak butuh penyimpanan yg b...,gamenya bagus dan nggak butuh penyimpanan yg b...
3,gm nya lebih bagus dari pada ml grafik hd,gm nya lebih bagus dari pada ml grafik hd
4,bagus game nya,bagus game nya
...,...,...
3711,game nya burik banget,game nya burik banget
3712,sampah. grafik doang. skil aneh ga guna. kesei...,sampah grafik doang skil aneh ga guna keseimba...
3713,masa dapat team kaya bocil kita yg pelanggaran...,masa dapat team kaya bocil kita yg pelanggaran...
3714,gem plgiat,gem plgiat


### Menghapus Huruf Berlebih
* Contoh : Mantappppp --> Mantap

In [ ]:
def normalize_word(word):
    return re.sub(r'(.)\1+', r'\1', word)

# Terapkan fungsi normalisasi pada kolom 'content'
data['clean_2'] = data['clean_1'].apply(normalize_word)

data[['clean_1', 'clean_2']]

,clean_1,clean_2
0,bagussssssssssss,bagus
1,lose streak mulu padahal masih gold musuh udah...,lose streak mulu padahal masih gold musuh udah...
2,gamenya bagus dan nggak butuh penyimpanan yg b...,gamenya bagus dan ngak butuh penyimpanan yg ba...
3,gm nya lebih bagus dari pada ml grafik hd,gm nya lebih bagus dari pada ml grafik hd
4,bagus game nya,bagus game nya
...,...,...
3711,game nya burik banget,game nya burik banget
3712,sampah grafik doang skil aneh ga guna keseimba...,sampah grafik doang skil aneh ga guna keseimba...
3713,masa dapat team kaya bocil kita yg pelanggaran...,masa dapat team kaya bocil kita yg pelangaran ...
3714,gem plgiat,gem plgiat


## Replacement (Normalisasi)

### Kamus Normalisasi

In [ ]:
norm = {
    'serusaya': 'seru saya',
    'byk': 'banyak',
    'terimakasih': 'terima kasih',
    'devloper': 'developer',
    'monton': 'moonton',
    'thank': 'terima kasih',
    'thanks': 'terima kasih',
    'dev': 'developer',
    'god': 'bagus',
    'mabar': 'main bareng',
    'ny': 'nya',
    'jga': 'juga',
    'ga': 'tidak',
    'dangk': 'dan tidak',
    'gk': 'tidak',
    'sat': 'saat',
    'yg': 'yang',
    'kang': 'abang',
    'deh': 'dah',
    'bwangwet': 'banget',
    'ni': 'ini',
    'bet': 'banget',
    'gravik': 'grafik',
    'geme': 'game',
    'mantab': 'mantap',
    'engak': 'tidak',
    'segansegan': 'segan',
    'colabepic': 'collab epic',
    'mainya': 'main nya',
    'enjoy': 'nyaman',
    'xixixixi': 'hahaha',
    'g': 'tidak',
    'skinya': 'skin nya',
    'gem': 'game',
    'gak': 'tidak',
    'sru': 'seru',
    'bru': 'baru',
    'apstore': 'appstore',
    'kikir': 'pelit',
    'legendaris': 'legends',
    'sgat': 'sangat',
    'games': 'game',
    'ok': 'oke',
    'freze' : 'freeze',
    'bgt': 'banget',
    'bgtbanyak': 'banget banyak',
    'kern': 'keren',
    'bgus': 'bagus',
    'gokil': 'keren',
    'nice': 'bagus',
    'fre': 'gratis',
    'tau': 'tahu',
    'smoth': 'halus',
    'bgs': 'bagus',
    'gam': 'game',
    'gil': 'gila',
    'nih': 'ini',
    'hehehehe': 'hahaha',
    'kren': 'keren',
    'kedepanya': 'kedepannya',
    'bagu': 'bagus',
    'downlond': 'unduh',
    'wow': 'luar biasa',
    'mantapmenghibur ': 'mantap menghibur',
    'say': 'saya',
    'sagat': 'sangat',
    'mlb': 'mlbb',
    'menyenang': 'menyenangkan',
    'kanmantap': 'mantap',
    'tuk': 'untuk',
    'gemnya': 'game nya',
    'anjir': 'anjing',
    'min': 'developer',
    'memainkanya': 'memainkannya',
    'sukak': 'suka',
    'semagat': 'semangat',
    'pokonya': 'pokoknya',
    'kaya': 'seperti',
    'mennag' : 'menang',
    'mnang' : 'menang',
    'kayak': 'seperti',
    'mengumkan': 'mengagumkan',
    'b': 'bintang',
    'rekomendet': 'direkomendasikan',
    'gacor': 'keren',
    'toxsi': 'kasar',
    'ngk': 'tidak',
    'kyk': 'seperti',
    'skin²': 'skin',
    'gameny': 'game nya',
    'baikm': 'baik',
    'dpat': 'dapat',
    'hero': 'hero',
    'bngt': 'banget',
    'lumyan': 'lumayan',
    'gemenya': 'game nya',
    'geme': 'game',
    'ska': 'suka',
    'lainya': 'lainnya',
    'mobilegend': 'mobile legends',
    'okeh': 'oke',
    'kek': 'seperti',
    'ges': 'teman',
    'bangt': 'banget',
    'trus': 'terus',
    'mantep': 'mantap',
    'drak': 'dark',
    'sytem': 'system',
    'sep': 'oke',
    'nyakarena': 'nya karena',
    'mudahsaya': 'mudah saya',
    'diamondsemoga': 'diamond semoga',
    'eventevent': 'event',
    'aja': 'saja',
    'gamenyaaku': 'game nya aku',
    'leg': 'lag',
    'nyah': 'nya',
    'lh': 'lah',
    'grafis': 'grafik',
    'mantao': 'mantap',
    'bro': 'kawan',
    'mantp': 'mantap',
    'jlas': 'jelas',
    'budeg': 'tidak mendengar',
    'kaga': 'tidak',
    'menangapi': 'menganggapi',
    'capek': 'lelah',
    'dm': 'diamond',
    'gilira': 'sedangkan',
    'giliran': 'sedangkan',
    'musu': 'musuh',
    'sosoan': 'sok',
    'ful': 'penuh',
    'dragsistem': 'dark system',
    'jaringansebab': 'jariingan sebab',
    'tingi': 'tinggi',
    'oke²aja': 'oke saja',
    'gj': 'tidak jelas',
    'ush': 'usah',
    'sistem': 'system',
    'gema': 'game',
    'gue': 'gw',
    'reng': 'rank',
    'susahlol': 'susah',
    'tin': 'tim',
    'sory': 'maaf',
    'uninstal': 'uninstall',
    'tpi': 'tapi',
    'ngeleg': 'ngelag',
    'ajaudah': 'saja sudah',
    'tolongsudah': 'tolong sudah',
    'stabilpadahal': 'stabil padahal',
    'kencengtapi': 'kencang tapi',
    'ngelagtolong': 'ngelag tolong',
    'draksistem': 'dark system',
    'jaringanya': 'jaringannya',
    'trlalu': 'terlalu',
    'pakek': 'pakai',
    'ngelak': 'ngelag',
    'gamae': 'game',
    'tu': 'itu',
    'dar': 'dark',
    'darksystemnya': 'dark system nya',
    'eror': 'error',
    'efect': 'efek',
    'klo': 'kalau',
    'loby': 'beranda',
    'logind': 'masuk',
    'maen': 'main',
    'lgsg': 'langsung',
    'pula': 'juga',
    'maenya': 'main nya',
    'niru' : 'plagiat',
    'meniru' : 'plagiat',
    'pelagiat': 'plagiat',
    'guoblok': 'goblok',
    'm': 'mm',
    'ajg': 'anjing',
    'kopi': 'niru',
    'plagiathapus': 'plagiat hapus',
    'tol': 'kontol',
    'c': 'cc',
    'nyengol': 'nyenggol',
    'sebelahtolong': 'sebelah tolong',
    'menjelekjelekan': 'menjelekkan',
    'respek': 'hormat',
    'gajelas': 'tidak jelas',
    'anj': 'anjing',
    'trol': 'troll',
    'muluarcher': 'selalu archer',
    'kalahmenang': 'kalah menang',
    'kalahyg': 'kalah yang',
    'turunmain': 'turun main',
    'lagend': 'legend',
    'tak': 'tidak',
    'd': 'di',
    'oada': 'pada',
    'poinya': 'poinnya',
    'mainmau': 'main mau',
    'ajh': 'saja',
    'bat': 'banget',
    'mengangu': 'mengganggu',
    'dowld': 'download',
    'pdh': 'padahal',
    'brp': 'berapa',
    'apl': 'aplikasi',
    'sistemnyacape': 'system nya cape',
    'buajingan': 'bajingan',
    'ku': 'aku',
    'stak': 'nyangkut',
    'dlu': 'dulu',
    'tetep': 'tetap',
    'gjls': 'tidak jelas',
    'pdhal': 'padahal',
    'asasin': 'assasin',
    'menyusahkantidak': 'menyusahkan tidak',
    'perbedaannyasama': 'perbedaannya sama',
    'blon': 'bodoh',
    'gtu': 'begitu',
    'enganya': 'engganya',
    'ben': 'beban',
    'heroskin': 'hero skin',
    'buriq': 'burik',
    'gamestolong': 'game tolong',
    'dinerfkarena': 'dinerf karena',
    'apaplkarena': 'apa karena',
    'anjdi': 'anjing di',
    'gd': 'tidak ada',
    'kntl': 'kontol',
    'anjg': 'anjing',
    'ngeleg²': 'ngelag',
    'apan': 'apaan',
    'spek': 'spesifikasi',
    'batle': 'perangg',
    'asw': 'anjing',
    'maf': 'maaf',
    'riloadkeluar': 'reload keluar',
    'mach': 'match',
    'rom': 'room',
    'oklah': 'oke lah',
    'ngak': 'tidak',
    'lagak': 'tidak',
    'masak': 'masa',
    'ajamatchmaking': 'saja matchmaking',
    'para': 'parah',
    'emel': 'ml',
    'as': 'anjing',
    'kntol': 'kontol',
    'ngetrol': 'ngetroll',
    'bayakan': 'banyakan',
    'f': 'ff',
    'mao': 'mau',
    'knp': 'kenapa',
    'sentiasa': 'senantiasa',
    'tolololol': 'tolol',
    'gm': 'game',
    'kadang²': 'kadang',
    'ngelek': 'ngelag',
    'myesel': 'kesal',
    'bocil': 'bocah kecil',
    'bnyak': 'banyak',
    'ngejiplak': 'plagiat',
    'pdhl': 'padahal',
    'sebelahmlb': 'sebelah mlbb',
    'dri': 'dari',
    'pda': 'pada',
    'bayak': 'banyak',
    'loding': 'loading',
    'kenceng': 'kencang',
    'permintan': 'permintaan',
    'satutolong': 'satu tolong',
    'diperbaikinfps': 'diperbaiki fps',
    'rankdan': 'rank dan',
    'namah': 'nama',
    'cinacoba': 'china coba',
    'bebepara': 'beberapa',
    'cina': 'china',
    'angota': 'anggota',
    'ama': 'sama',
    'frez': 'freeze',
    'iklanya': 'iklan nya',
    'penjelasanya': 'penjelasannya',
    'tdk': 'tidak',
    'anjim': 'anjing',
    'geblek': 'goblok',
    'losetreak': 'losestreak',
    'kerjan': 'kerjaan',
    'gebleg': 'goblok',
    'jgn': 'jangan',
    'ngefrane': 'ngeframe',
    'folow': 'ikut',
    'fidio': 'vidio',
    'tlol': 'tolol',
    'warasmasa': 'waras masa',
    'diapainditambah': 'diapain ditambah',
    'ngoptimalkan': 'mengoptimalkan',
    'susahpakai': 'susah pakai',
    'hutankocak': 'hutan kocak',
    'koplak': 'kocak',
    'nob': 'payah',
    'bangat' : 'banget',
    'elek': 'jelek',
    'banga': 'bangga',
    'bae': 'aja',
    'mnding': 'mending',
    'minuntuk': 'developer untuk',
    'pecahkansama': 'pecahkan sama',
    'otomatispaling': 'otomatis paling',
    'susuah': 'susah',
    'gamenyabaru': 'game nya baru',
    'gamenyapadahal': 'game nya padahal',
    'dmn': 'dimana',
    'pd': 'pada',
    'patahsinyal': 'patah sinyal',
    'tibatiba': 'tiba',
    'aru': 'baru',
    'lgi': 'lagi',
    'yabiar': 'ya biar',
    'mainyasama': 'main nya sama',
    'kalu': 'kalau',
    'menantangsatu': 'menantang satu',
    'suman': 'cuman',
    'sihgrafik': 'sih grafik',
    'iyo': 'iya',
    'terbitkasian': 'terbit kasian',
    'nga': 'tidak',
    'ngebag': 'ngebug',
    'ajah': 'saja',
    'manding': 'mending',
    'stafstaf': 'staf',
    'kont': 'kontol',
    'ngeleq': 'ngelag',
    'gaje': 'tidak jelas',
    'goblog': 'goblok',
    'aga': 'agak',
    'benarbenar': 'benar',
    'setres': 'setress',
    'darksistem': 'dark system',
    'mlbedakelasbedakualitas': '',
    'nyingung': 'nyinggung',
    'asu': 'anjing',
    'dwld': 'download',
    'kagak': 'tidak',
    'ngestack': 'ngestuck',
    'tampat': 'tempat',
    'cmn': 'cuman',
    'lalukek': 'lalu seperti',
    'sm': 'sama',
    'diamon': 'diamond',
    'ngestak': 'ngestuck',
    'matakata': 'mata kata',
    'yasiapa': 'ya siapa',
    'lagih': 'lagi',
    'judescuek': 'judes cuek',
    'pengunan': 'penggunaan',
    'bnget': 'banget',
    'semuagame': 'semua game',
    'kakulahnyaga': 'kaku lagnya tidak',
    'mainmasa': 'main masa',
    'gamegame': 'game',
    'sbelah': 'sebelah',
    'awak': 'awal',
    'gerakanya': 'gerakannya',
    'blok': 'goblok',
    'mingu': 'minggu',
    'kekgini': 'kayak begini',
    'emng': 'emang',
    'jaringansoalnya': 'jaringan soalnya',
    'mainpingnya': 'main pingnya',
    'capekintinya': 'lelah intinya',
    'baikitu': 'baik itu',
    'ngeleq' : 'ngelag',
    'stuntombol': 'stuntombol',
    'setalah': 'setelah',
    'stuk': 'stuck',
    'jengkelmemuat': 'kesal loading',
    'jengkel' : 'kesal',
    'memuat' : 'loading',
    'stack': 'stuck',
    'legen': 'legends',
    'ak': 'aku',
    'sekalikalah': 'sekali kalah',
    'sesesui': 'sesuai',
    'karna': 'karena',
    'gax': 'tidak',
    'memengkan': 'memenangkan',
    'brkurang': 'berkurang',
    'tangapi': 'tanggapi',
    'kurangloby': 'kurang beranda',
    'legens': 'legends',
    'bangr': 'banget',
    'mkn': 'makan',
    'ngfrane': 'ngeframe',
    'bnr': 'benar',
    'ketingalan': 'ketinggalan',
    'matchmaching': 'match making',
    'majapahitkaku': 'majapahit kaku',
    'bong': 'bohong',
    'guna': 'berguna',
    'sma': 'sama',
    'udh': 'udah',
    'darksystem': 'dark system',
    'riload': 'reload',
    'kesukan': 'kesukaan',
    'gemanya': 'game nya',
    'mobilr': 'mobile',
    'hadeh': 'aduh',
    'ngabissin': 'ngehabisin',
    'lo': 'lu',
    'ntap': 'mantap',
    'legand': 'legend',
    'seberu': 'seru',
    'mantaf': 'mantap',
    'dabest': 'terbaik',
    'bersemqngat': 'bersemangat',
    'feder': 'feeder',
    'ilang': 'hilang',
    'asek': 'asik',
    'mantul': 'mantap betul',
    'pnyakitnya': 'penyakitnya',
    'sebelumnyamlserver': 'sebelumnya ml server',
    'cacad': 'cacat',
    'develover': 'developer',
    'anjinx': 'anjing',
    'muludapat': 'mulu dapat',
    'ampasnaik': 'ampas naik',
    'tertingi': 'tertinggi',
    'geratis': 'gratis',
    'ifen': 'event',
    'pintng': 'bintang',
    'jdi': 'jadi',
    'tencentbaek': 'tencent baik',
    'fed': 'feed',
    'cmantep': 'mantap',
    'nij': 'ini',
    'bagusgrafik': 'bagus grafik',
    'lancarhero': 'lancar hero',
    'demage': 'damage',
    'turet': 'turret',
    'tp': 'tapi',
    'lemot': 'lama',
    'kasi': 'kasih',
    'kntil': 'kontol',
    'bodo': 'bodoh',
    'moga': 'semoga',
    'menaikan': 'menaikkan',
    'ke bayakan': 'kebanyakan',
    'idamanmanjain': 'idaman manjain',
    'kingsmemang': 'kings memang',
    'ngedadak': 'mendadak',
    'systim': 'system',
    'mls': 'malas',
    'best': 'terbaik',
    'serulah': 'seru lah',
    'bagos': 'bagus',
    'deley': 'delay',
    'ofline': 'offline',
    'kok': 'malah',
    'percumah': 'percuma',
    'ap': 'apa',
    'cba': 'coba',
    'kenakan': 'keenakan',
    'dog': 'anjing',
    'mobilegends': 'mobile legends',
    'maenin': 'mainin',
    'rengkednya': 'ranked nya',
    'bner': 'benar',
    'marchmackingnya': 'matchmaking nya',
    'bruntun': 'beruntun',
    'mntap': 'mantap',
    'bagus san': 'bagusan',
    'gada': 'tidak ada',
    'reting': 'rating',
    'nyindar': 'nyindir',
    'ngeprime': 'ngeframe',
    'okemackmaking': 'oke matchmaking',
    'nabrakbaru': 'nabrak baru',
    'sebelahmobile': 'sebelah mobile',
    'muda': 'mudah',
    'copy': 'plagiat',
    'tungu': 'tunggu',
    'bagussaya': 'bagus saya',
    'perainanya': 'permainannya',
    'inisalam': 'ini salam',
    'the best': 'terbaik',
    'gp': 'gapapa',
    'kil': 'kill',
    'recal': 'recall',
    'skil': 'skill',
    'kumainin': 'aku mainin',
    'perkembangkanlebih': 'berkembang lebih',
    'perkembangkan': 'berkembang',
    'baikramah': 'baik ramah',
    'dn': 'dan',
    'bnyk': 'banyak',
    'mlh': 'malah',
    'gx': 'tidak',
    'serujuga': 'serujuga',
    'dpt': 'dapat',
    'elu': 'lu',
    'setu': 'seru',
    'jlk': 'jelek',
    'gim' : 'game',
    'gangu': 'ganggu',
    'aj': 'aja',
    'gimna': 'gimana',
    'plagiyat': 'plagiat',
    'slain': 'selain',
    'sagt': 'sangat',
    'lg' : 'lagi',
    'kalo': 'kalau',
    'gemes': 'game',
    'bantuanya': 'bantuannya',
    'muluk': 'mulu',
    'sprt': 'seperti',
    'scren': 'screen',
    'mobilelegend': 'mobile legend',
    'garfik': 'grafik',
    'jagi': 'jago',
    'anjeng' : 'anjing',
    'jngan': 'jangan',
    'moton': 'moonton',
    'lbh': 'lebih',
    'drpd': 'daripada',
    'semingu': 'seminggu',
    'overal': 'keseluruhan',
    'gemanya' : 'game nya',
    'skli' : 'sekali',
    'ngefreze' : 'freeze',
    'systm': 'system',
    'gemplagiat': 'game plagiat',
    'plgiat' : 'plagiat',
    'ktmu' : 'ketemu',
    'ketmu' : 'ketemu',
    'lumayam': 'lumayan',
    'lumayamlah': 'lumayan lah',
    'ngbug' : 'ngebug',
    'hok' : 'honor of kings',
    'langen': 'legends',
    'palagiat': 'plagiat',
    'matcmaking' : 'matchmaking',
    'ng': 'tidak',
    'jiplak' : 'plagiat',
    'penjiplak' : 'plagiat',
    'sistemnyabikin' : 'system nya membuat',
    'bikin' : 'membuat',
    'daksistem' : 'dark system',
    'dak' : 'dark',
    'sayasistem' : 'saya system',
    'sistemnya' : 'system nya'
 }

### Fungsi Normalisasi

In [ ]:
# Fungsi untuk normalisasi kata
def normalize_text(text):
    words = text.split()  # Memisahkan kata
    normalized_words = [norm.get(word, word) for word in words]
    return ' '.join(normalized_words)  # Gabungkan kembali menjadi string


data['normalized'] = data['clean_2'].apply(normalize_text)

# Tampilkan hasil
data[['clean_2' , 'normalized']]

,clean_2,normalized
0,bagus,bagus
1,lose streak mulu padahal masih gold musuh udah...,lose streak mulu padahal masih gold musuh udah...
2,gamenya bagus dan ngak butuh penyimpanan yg ba...,gamenya bagus dan tidak butuh penyimpanan yang...
3,gm nya lebih bagus dari pada ml grafik hd,game nya lebih bagus dari pada ml grafik hd
4,bagus game nya,bagus game nya
...,...,...
3711,game nya burik banget,game nya burik banget
3712,sampah grafik doang skil aneh ga guna keseimba...,sampah grafik doang skill aneh tidak berguna k...
3713,masa dapat team kaya bocil kita yg pelangaran ...,masa dapat team seperti bocah kecil kita yang ...
3714,gem plgiat,game plagiat


In [ ]:
# Filter baris yang mengandung kata 'sistem' dalam kolom 'normalized'
filtered_data = data[data['normalized'].str.contains('gemanya', case=False, na=False)]

# Menampilkan hasil
filtered_data[['normalized']]

,normalized


## Tokenize

In [ ]:
data['tokens'] = data['normalized'].apply(word_tokenize)

data[['normalized', 'tokens']]

,normalized,tokens
0,bagus,[bagus]
1,lose streak mulu padahal masih gold musuh udah...,"[lose, streak, mulu, padahal, masih, gold, mus..."
2,gamenya bagus dan tidak butuh penyimpanan yang...,"[gamenya, bagus, dan, tidak, butuh, penyimpana..."
3,game nya lebih bagus dari pada ml grafik hd,"[game, nya, lebih, bagus, dari, pada, ml, graf..."
4,bagus game nya,"[bagus, game, nya]"
...,...,...
3711,game nya burik banget,"[game, nya, burik, banget]"
3712,sampah grafik doang skill aneh tidak berguna k...,"[sampah, grafik, doang, skill, aneh, tidak, be..."
3713,masa dapat team seperti bocah kecil kita yang ...,"[masa, dapat, team, seperti, bocah, kecil, kit..."
3714,game plagiat,"[game, plagiat]"


## Menghapus Stopword

In [ ]:
# Daftar stopword dari NLTK (bahasa Indonesia)
stop_words = set(stopwords.words('indonesian'))

# Kata yang ingin dikecualikan dari stopword (misalnya 'tidak' dan 'baik' tidak ingin dihapus)
exclude_words = {
    'tidak',
    'baik',
    'bisa',
    'kecil',
    'semuanya',
    'banyak',
    'jelas',
    'mirip',
    'lebih',
    'sangat',
    'semua',
    'jauh',
    'jangan',
    'naik',
    'tepat',
    'luar',
    'baru',
    'dapet',
    'pernah',
    'benar',
    'masalah',
    'buat',
    'sering',
    'punya',
    'sekali',
    'keluar',
    'mulai',
    'percuma'
}

# Kata yang ingin dimasukkan ke dalam daftar stopword
include_words = {
    'nya',
    'gw',
    'gua',
    'lu'
}

# Hapus kata-kata yang dikecualikan dari daftar stopword
stop_words = stop_words.difference(exclude_words)

# Tambahkan kata-kata yang ingin disertakan ke dalam stopword
stop_words = stop_words.union(include_words)

# Fungsi untuk menghapus stopword
def remove_stopwords(tokens, stop_words):
    return [token for token in tokens if token not in stop_words]

# Terapkan filtering stopword pada kolom 'tokens'
data['stopword'] = data['tokens'].apply(lambda x: remove_stopwords(x, stop_words))

# Tampilkan hasil
data[['tokens', 'stopword']]

,tokens,stopword
0,[bagus],[bagus]
1,"[lose, streak, mulu, padahal, masih, gold, mus...","[lose, streak, mulu, gold, musuh, udah, susah,..."
2,"[gamenya, bagus, dan, tidak, butuh, penyimpana...","[gamenya, bagus, tidak, butuh, penyimpanan, ba..."
3,"[game, nya, lebih, bagus, dari, pada, ml, graf...","[game, lebih, bagus, ml, grafik, hd]"
4,"[bagus, game, nya]","[bagus, game]"
...,...,...
3711,"[game, nya, burik, banget]","[game, burik, banget]"
3712,"[sampah, grafik, doang, skill, aneh, tidak, be...","[sampah, grafik, doang, skill, aneh, tidak, be..."
3713,"[masa, dapat, team, seperti, bocah, kecil, kit...","[team, bocah, kecil, pelangaran, hade]"
3714,"[game, plagiat]","[game, plagiat]"


In [ ]:
# Fungsi untuk menghapus stopword
# Fungsi untuk menghapus stopword dan mencatat stopword yang dihapus
def remove_stopwords_with_tracking(tokens, stop_words):
    removed_stopwords = []  # List untuk menyimpan kata yang dihapus
    filtered_tokens = []    # List untuk menyimpan token setelah stopword dihapus

    for token in tokens:
        if token in stop_words:
            removed_stopwords.append(token)  # Simpan stopword yang dihapus
        else:
            filtered_tokens.append(token)    # Simpan token yang bukan stopword

    return filtered_tokens, removed_stopwords

# Terapkan filtering stopword pada kolom 'tokens' dan simpan stopword yang dihapus
data['stop'] = data['tokens']
data['cek_stopword'], data['removed_stopwords'] = zip(*data['stop'].apply(lambda x: remove_stopwords_with_tracking(x, stop_words)))

# Tampilkan hasil
data[['stop', 'cek_stopword', 'removed_stopwords']]

,stop,cek_stopword,removed_stopwords
0,[bagus],[bagus],[]
1,"[lose, streak, mulu, padahal, masih, gold, mus...","[lose, streak, mulu, gold, musuh, udah, susah,...","[padahal, masih]"
2,"[gamenya, bagus, dan, tidak, butuh, penyimpana...","[gamenya, bagus, tidak, butuh, penyimpanan, ba...","[dan, yang, daripada, jadi, untuk, di, saya, l..."
3,"[game, nya, lebih, bagus, dari, pada, ml, graf...","[game, lebih, bagus, ml, grafik, hd]","[nya, dari, pada]"
4,"[bagus, game, nya]","[bagus, game]",[nya]
...,...,...,...
3711,"[game, nya, burik, banget]","[game, burik, banget]",[nya]
3712,"[sampah, grafik, doang, skill, aneh, tidak, be...","[sampah, grafik, doang, skill, aneh, tidak, be...","[pada, di, saat]"
3713,"[masa, dapat, team, seperti, bocah, kecil, kit...","[team, bocah, kecil, pelangaran, hade]","[masa, dapat, seperti, kita, yang]"
3714,"[game, plagiat]","[game, plagiat]",[]


### Pengecekan stopword

In [ ]:
# Download stopwords jika belum diunduh
nltk.download('stopwords')

# Ambil daftar stopword bahasa Indonesia
stop_words_indonesian = stopwords.words('indonesian')

# Fungsi untuk mengecek apakah kata ada di stopword
def is_stopword(word):
    return word in stop_words_indonesian

# Contoh penggunaan
x = 'saya'
if is_stopword(x):
    print(f"'{x}' adalah stopword")
else:
    print(f"'{x}' bukan stopword")


'saya' adalah stopword


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Stemming

In [145]:
# Membuat objek stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi stemming
def stemming_indonesia(tokens):
    return [stemmer.stem(token) for token in tokens]

data['stemming'] = data['stopword'].apply(stemming_indonesia)

data[['stopword','stemming']]

,stopword,stemming
0,[bagus],[bagus]
1,"[lose, streak, mulu, gold, musuh, udah, susah,...","[lose, streak, mulu, gold, musuh, udah, susah,..."
2,"[gamenya, bagus, tidak, butuh, penyimpanan, ba...","[gamenya, bagus, tidak, butuh, simpan, banyak,..."
3,"[game, lebih, bagus, ml, grafik, hd]","[game, lebih, bagus, ml, grafik, hd]"
4,"[bagus, game]","[bagus, game]"
...,...,...
3711,"[game, burik, banget]","[game, burik, banget]"
3712,"[sampah, grafik, doang, skill, aneh, tidak, be...","[sampah, grafik, doang, skill, aneh, tidak, gu..."
3713,"[team, bocah, kecil, pelangaran, hade]","[team, bocah, kecil, pelangaran, hade]"
3714,"[game, plagiat]","[game, plagiat]"


In [146]:
# Fungsi stemming dan mencatat stopword yang dihapus
def stemming_indonesia_with_stopwords(tokens):
    removed_stopwords = []  # List untuk menyimpan kata stopword yang dihapus
    stemmed_tokens = []     # List untuk menyimpan hasil stemming

    for token in tokens:
        try:
            stemmed_token = stemmer.stem(token)  # Proses stemming pada setiap token (kata)

            if token != stemmed_token:  # Jika token berubah setelah stemming
                removed_stopwords.append(token)  # Masukkan ke list stopword yang dihapus

            stemmed_tokens.append(stemmed_token)  # Tambahkan token hasil stemming ke list
        except Exception as e:
            print(f"Error stemming token: {token}, {e}")  # Log error jika ada masalah dengan token tertentu

    return stemmed_tokens, removed_stopwords

# Terapkan stemming pada kolom 'stopword' dan simpan stopword yang dihapus
data['stem'] = data['stopword']
data['cek_stemming'], data['removed_stemming'] = zip(*data['stopword'].apply(stemming_indonesia_with_stopwords))

# Menampilkan hasil
data[['stem', 'stemming', 'removed_stemming']]


,stem,stemming,removed_stemming
0,[bagus],[bagus],[]
1,"[lose, streak, mulu, gold, musuh, udah, susah,...","[lose, streak, mulu, gold, musuh, udah, susah,...",[]
2,"[gamenya, bagus, tidak, butuh, penyimpanan, ba...","[gamenya, bagus, tidak, butuh, simpan, banyak,...","[penyimpanan, mainkan]"
3,"[game, lebih, bagus, ml, grafik, hd]","[game, lebih, bagus, ml, grafik, hd]",[]
4,"[bagus, game]","[bagus, game]",[]
...,...,...,...
3711,"[game, burik, banget]","[game, burik, banget]",[]
3712,"[sampah, grafik, doang, skill, aneh, tidak, be...","[sampah, grafik, doang, skill, aneh, tidak, gu...","[berguna, keseimbangan, pemainya, mengerikan]"
3713,"[team, bocah, kecil, pelangaran, hade]","[team, bocah, kecil, pelangaran, hade]",[]
3714,"[game, plagiat]","[game, plagiat]",[]


In [147]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3716 entries, 0 to 3715
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   content            3716 non-null   object 
 1   sentimen           3716 non-null   float64
 2   lower_case         3716 non-null   object 
 3   clean_1            3716 non-null   object 
 4   clean_2            3716 non-null   object 
 5   normalized         3716 non-null   object 
 6   tokens             3716 non-null   object 
 7   stopword           3716 non-null   object 
 8   stop               3716 non-null   object 
 9   cek_stopword       3716 non-null   object 
 10  removed_stopwords  3716 non-null   object 
 11  stemming           3716 non-null   object 
 12  stem               3716 non-null   object 
 13  cek_stemming       3716 non-null   object 
 14  removed_stemming   3716 non-null   object 
dtypes: float64(1), object(14)
memory usage: 464.5+ KB


In [148]:
data.isna().sum()

,0
content,0
sentimen,0
lower_case,0
clean_1,0
clean_2,0
normalized,0
tokens,0
stopword,0
stop,0
cek_stopword,0


In [ ]:
data[['content','stemming','sentimen']]

### Nyoba stemming satu kalimat

In [ ]:
# coba stemming 1 kalimat (buat nyoba)
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi stemming
def stemming_indonesia(text):
    return stemmer.stem(text)

# Contoh penggunaan
kalimat = "permainan ini menarik"
hasil_stemming = stemming_indonesia(kalimat)

print(hasil_stemming)

In [ ]:
data = data[['content','stemming', 'sentimen']]

In [ ]:
data.isna().sum()

# Modeling Word2Vec

In [ ]:
X = data['stemming']  # features (text data)
y = data['sentimen']  # labels (sentimen)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
test = X_test.copy()
test

In [ ]:
sentences = X_train
sentences

In [ ]:
model = Word2Vec(sentences=sentences, vector_size=128, window=3, min_count=3, workers=4, epochs=100, sg=1, hs=0,negative = 5)

In [ ]:
word = model.wv

In [ ]:
word.most_similar('honor')

In [ ]:
vektor = word['honor']
print(vektor)

In [ ]:
os.makedirs('model', exist_ok=True)
model.save('model/word2vec_best.w2v')

In [ ]:
X_trainc = X_train.copy()
X_testc = X_test.copy()
y_trainc = y_train.copy()
y_testc = y_test.copy()

In [ ]:
word_counts = [len(tokens) for tokens in X_trainc]

max_word_count = max(word_counts)

print(f"Jumlah kata terbanyak: {max_word_count}")

# Modeling LSTM

In [ ]:
word2vec_model = Word2Vec.load('word2vec_best.w2v')
w2v = word2vec_model.wv

In [ ]:
max_sequence_length = 70

def convert_and_pad(data, word2vec_model, maxlen, vector_size):
    vectors = [
        [word2vec_model[word] if word in word2vec_model else np.zeros(vector_size) for word in sentence]
        for sentence in data
    ]
    vectors_padded = pad_sequences(vectors, maxlen=maxlen, padding='post', dtype='float32')
    return vectors_padded

X_train_vectors = convert_and_pad(X_trainc, w2v, max_sequence_length, vector_size=128)
X_test_vectors = convert_and_pad(X_testc, w2v, max_sequence_length, vector_size=128)

print("Shape of X_train_padded:", X_train_vectors.shape)
print("Shape of X_test_padded:", X_test_vectors.shape)

In [ ]:
X_trainc.iloc[0]

In [ ]:
vektor = w2v['seru']
vektor

In [ ]:
X_train_vectors[0][0]

In [ ]:
max_length = max(len(vec) for vec in X_train_vectors)
word2vec_vector_size = 128

# LSTM model
lstm = Sequential()
lstm.add(Input(shape=(max_length, word2vec_vector_size)))
lstm.add(LSTM(units=128))
lstm.add(Dropout(0.5))
lstm.add(Dense(1, activation='sigmoid'))

# Compile the model
lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
X_train_vectorsc = X_train_vectors.copy()
y_traincc = y_trainc.copy()

In [ ]:
lstm.fit(X_train_vectorsc, y_traincc, epochs=15, batch_size=32, validation_split=0.2)

# Evaluasi

In [ ]:
test_loss, test_accuracy = lstm.evaluate(X_test_vectors, y_testc)
print(f'Test accuracy: {test_accuracy}')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score,f1_score, roc_auc_score, classification_report, confusion_matrix, precision_recall_curve
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

def eval_classification(model, x_tra, x_tes,
                       y_tra, y_tes, thres):

    # Menggunakan model.predict() untuk mendapatkan prediksi
    y_pred_train = model.predict(x_tra)
    y_pred_proba_train = model.predict(x_tra)

    # Mendapatkan probabilitas untuk data tes
    y_pred_proba = model.predict(x_tes)

    # Jika model adalah untuk klasifikasi biner, gunakan threshold untuk mendapatkan prediksi kelas
    threshold = thres
    y_pred = (y_pred_proba >= threshold).astype('int')

    # Menampilkan hasil evaluasi
    print(classification_report(y_tes, y_pred))
    print("Accuracy (Test Set): %.2f" % accuracy_score(y_tes, y_pred))
    print("Precision (Test Set): %.2f" % precision_score(y_tes, y_pred))
    print("Recall (Test Set): %.2f" % recall_score(y_tes, y_pred))
    print("F1-Score (Test Set): %.2f" % f1_score(y_tes, y_pred))
    print("roc_auc (Test-proba): %.2f" % roc_auc_score(y_tes, y_pred_proba))
    print("roc_auc (Train-proba): %.2f" % roc_auc_score(y_tra, y_pred_proba_train))

    print(f"Confusion Matrix: \n{confusion_matrix(y_tes, y_pred)}\n")


In [ ]:
eval_classification(lstm, X_train_vectorsc, X_test_vectors, y_traincc, y_testc, 0.5)

In [ ]:
def eval_classification(model, X_train, X_test, y_train, y_test, threshold=0.5):
    # Fit the model

    # Predict probabilities and convert to binary predictions based on threshold
    y_pred_proba = model.predict(X_test).flatten()  # Assuming binary classification
    y_pred = (y_pred_proba >= threshold).astype(int)

    # Create a DataFrame with actual sentiments, predictions, and prediction categories
    test_results = pd.DataFrame({
        'Actual Sentiment': y_test,
        'Predicted Sentiment': y_pred,
        'Prediction Probability': y_pred_proba
    })

    # Define TF, TP, FN, FP based on comparison between actual and predicted sentiments
    conditions = [
        (test_results['Actual Sentiment'] == 1) & (test_results['Predicted Sentiment'] == 1),
        (test_results['Actual Sentiment'] == 0) & (test_results['Predicted Sentiment'] == 0),
        (test_results['Actual Sentiment'] == 0) & (test_results['Predicted Sentiment'] == 1),
        (test_results['Actual Sentiment'] == 1) & (test_results['Predicted Sentiment'] == 0)
    ]
    choices = ['TP', 'TN', 'FP', 'FN']
    test_results['Category'] = np.select(conditions, choices, default='Unknown')

    # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Plotting the confusion matrix
    plt.figure(figsize=(4, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Predicted Negative', 'Predicted Positive'],
                yticklabels=['Actual Negative', 'Actual Positive'])
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

    return test_results

# Memanggil fungsi
test_results = eval_classification(lstm, X_train_vectorsc, X_test_vectors, y_traincc, y_testc, 0.5)

In [ ]:
# Save model
keras.saving.save_model(lstm, 'LSTM_94.keras')